# Fisher test

In [1]:
import os, sys, time
import numpy as np
import scipy as sp
import healpy as hp
from fgbuster.observation_helpers import *

import micmac

In [2]:
working_directory_path = os.path.abspath('') + '/'
# directory_save_file = working_directory_path + 'save_directory/'
directory_toml_file = working_directory_path #+ 'toml_params/'
path_toml_file = directory_toml_file + 'full_chain_v1a.toml'
# path_toml_file = directory_toml_file + 'LiteBIRD_full_chain_v1a.toml'

In [3]:
MICMAC_sampler_obj = micmac.create_MICMAC_sampler_from_toml_file(path_toml_file)

In [4]:
# General parameters
cmb_model = 'c1'
cmb_model = ''
fgs_model = 'd0s0'
model = cmb_model+fgs_model
noise = True
# noise = False
noise_seed = 42
# instr_name = dictionary_parameters['instrument_name'] #'SO_SAT'
# instr_name = 'SO_SAT'
instr_name = MICMAC_sampler_obj.instrument_name

# get instrument from public database
instrument = get_instrument(instr_name)

# get input freq maps
np.random.seed(noise_seed)
# freq_maps = get_observation(instrument, model, nside=NSIDE, noise=noise)[:, 1:, :]   # keep only Q and U
freq_maps_fgs = get_observation(instrument, fgs_model, nside=MICMAC_sampler_obj.nside, noise=noise)[:, 1:, :]   # keep only Q and U
print("Shape for input frequency maps :", freq_maps_fgs.shape)

fgbuster_input_freq_maps = get_observation(instrument, model, nside=MICMAC_sampler_obj.nside, noise=noise)[:, 1:, :]   # keep only Q and U

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Shape for input frequency maps : (6, 2, 49152)


In [5]:
init_mixing_matrix_obj = micmac.InitMixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)

exact_params_mixing_matrix = init_mixing_matrix_obj.init_params()

mixing_matrix_obj = micmac.MixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, exact_params_mixing_matrix, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)

inv_Nl = micmac.get_true_Cl_noise(np.array(instrument['depth_p']), lmax=MICMAC_sampler_obj.lmax)

# Getting input data

In [6]:
input_freq_maps, input_cmb_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_sampler_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)

theoretical_r0_total = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)#[partial_indices_polar,:]
theoretical_r1_tensor = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)#[partial_indices_polar,:]

Calculating spectra from CAMB !
Calculating spectra from CAMB !


In [7]:
# data_map = input_freq_maps
data_map = fgbuster_input_freq_maps

data_map_extended = np.insert(data_map, [0], np.zeros(MICMAC_sampler_obj.npix), axis=1)

In [8]:
indices_polar = np.array([1,2,4])
partial_indices_polar = indices_polar[:MICMAC_sampler_obj.nstokes]

c_ell_approx = np.zeros((6, MICMAC_sampler_obj.lmax+1))
c_ell_approx[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r0_total

Cl_lens = np.zeros_like(c_ell_approx)
Cl_lens[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r0_total

Cl_prim = np.zeros_like(c_ell_approx)
Cl_prim[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r1_tensor

# Compute Fisher

In [9]:
mode_Fisher = 'EB'

In [10]:
Fisher_matrix = micmac.full_fisher(mixing_matrix_obj, exact_params_mixing_matrix, Cl_lens, Cl_prim, data_map_extended, MICMAC_sampler_obj.lmax, inv_Nl, 
                            lmin=MICMAC_sampler_obj.lmin, r_start=MICMAC_sampler_obj.r_true, mode=mode_Fisher, Sc_approx=c_ell_approx)

1 of 6 complete
2 of 6 complete
3 of 6 complete
4 of 6 complete
5 of 6 complete
6 of 6 complete


Computing mixing matrix
A_dB =  [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


/Users/mag/miniconda3/envs/non_param_silver/lib/python3.9/site-packages/micmac/harmonic_Fisher_tools.py:119: RuntimeWarning: invalid value encountered in divide
  m1_dBdB = 2.0*np.trace(_mtmm(A_dB_i, P, _mm(A_dB_j, sst_avg)), axis1=-1, axis2=-2) - Nc_dB*Nc_dB_prime/((Sc+Nc)**2) + Nc_dB*Nc_dB_prime/((Sc_approx+Nc)**2)
/Users/mag/miniconda3/envs/non_param_silver/lib/python3.9/site-packages/micmac/harmonic_Fisher_tools.py:120: RuntimeWarning: invalid value encountered in divide
  m2_dBdB = 2.0*(Sc-Sc_approx)/((Sc_approx+Nc)*(Sc+Nc))*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), cov, _mtmm(A, invN, A_dB_j))
/Users/mag/miniconda3/envs/non_param_silver/lib/python3.9/site-packages/micmac/harmonic_Fisher_tools.py:124: RuntimeWarning: invalid value encountered in multiply
  m3_dBdB = 2.0/(Sc+Nc)*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), sst_avg, _mtmm(A_dB_j, invN, A)), cov)[..., 0, 0]


A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
[[ 2.11294845e+08 -1.84053066e+08  2.36362583e+07  1.03049983e+07
   3.09774094e+08 -2.66268805e+08  3.64434174e+07  1.47995640e+07
   0.00000000e+00]
 [-1.84053066e+08  5.95930825e+09 -5.76755365e+09 -9.70524643e+06
  -2.66273432e+08  8.47907451e+09 -8.58516926e+09  8.07472250e+06
   0.00000000e+00]
 [ 2.36362583e+07 -5.76755365e+09  7.03491238e+09 -1.04548428e+09
   3.64413571e+07 -8.58518565e+09  1.02520183e+10 -1.52197637e+09

/Users/mag/miniconda3/envs/non_param_silver/lib/python3.9/site-packages/micmac/harmonic_Fisher_tools.py:132: RuntimeWarning: invalid value encountered in divide
  m_drdr = (Sc_dr/(Sc+Nc))**2


In [11]:
sqrt_inv_Fisher = sp.linalg.sqrtm(np.linalg.pinv(Fisher_matrix))

In [12]:
np.diag(sqrt_inv_Fisher)

array([1.22719235e-04, 4.72984090e-05, 4.44084763e-05, 5.23546882e-05,
       4.28340683e-05, 1.62270766e-05, 1.52905384e-05, 1.82529746e-05,
       5.55036342e-04])

In [16]:
path_dir_Fisher = '/Users/mag/Documents/PHD1Y/Space_Work/Pixel_non_P2D/MICMAC/test_playground/'

lmin = MICMAC_sampler_obj.lmin
lmax = MICMAC_sampler_obj.lmax
r_true = MICMAC_sampler_obj.r_true
name_Fisher = 'Fisher_matrix_'+'instr_name'+mode_Fisher+'_model_'+model+'_noise_'+str(noise)+'_seed_'+str(noise_seed)+f'_lmin{lmin}_lmax{lmax}'+'.txt'

print('Saving Fisher matrix in', path_dir_Fisher + name_Fisher)
np.savetxt(path_dir_Fisher + name_Fisher, Fisher_matrix)


Saving Fisher matrix in /Users/mag/Documents/PHD1Y/Space_Work/Pixel_non_P2D/MICMAC/test_playground/Fisher_matrix_instr_nameEB_model_d0s0_noise_True_seed_42_lmin2_lmax128.txt
